In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm
import random

In [41]:
#read data
testdata = pd.read_csv('data/test.csv',sep='\t')
train = pd.read_csv('data/train.csv',sep='\t')
newTest = testdata.dropna(subset=['Correct First Attempt'])

In [42]:
#remove time data and "Incorrects", "Hints", "Corrects"
def drop_unusablelables(x):
    a = x.drop(['Row','Correct Transaction Time', 'Step End Time','Step Duration (sec)', 'First Transaction Time',\
                'Step Start Time','Correct Step Duration (sec)','Hints','Corrects','Error Step Duration (sec)',\
                'Incorrects'],axis=1)
    return a
test = drop_unusablelables(testdata)

pretrain = drop_unusablelables(train)

pretest = drop_unusablelables(newTest)

In [43]:
#calculate studentCFAR
def CFAR(x):
    return studentCFAR[x]
studentCFAR = pretrain.groupby('Anon Student Id')['Correct First Attempt'].agg(np.mean)
test['CFAR_student'] = test['Anon Student Id'].apply(CFAR)


pretrain['CFAR_student'] = pretrain['Anon Student Id'].apply(CFAR)


pretest['CFAR_student'] = pretest['Anon Student Id'].apply(CFAR)

In [44]:
#caculate stepCFAR
stepCFAR = pretrain.groupby('Step Name')['Correct First Attempt'].agg(np.mean)

def pretrain_stepCFAR(x):
    return stepCFAR[x]


def step_CFAR(x):
    if x in stepCFAR.keys():
        return stepCFAR[x]
    else:
        return stepCFAR.mean()

test['CFAR_step'] = test['Step Name'].apply(step_CFAR)

pretrain['CFAR_step'] = pretrain['Step Name'].apply(pretrain_stepCFAR)

pretest['CFAR_step'] = pretest['Step Name'].apply(step_CFAR)


In [45]:
#caculate problemCFAR
problemCFAR = pretrain.groupby('Problem Name')['Correct First Attempt'].agg(np.mean)

def pretrain_problemCFAR(x):
    return problemCFAR[x]


def problem_CFAR(x):
    if x in problemCFAR.keys():
        return problemCFAR[x]
    else:
        return problemCFAR.mean()
test['CFAR_problem'] = test['Problem Name'].apply(problem_CFAR)

pretrain['CFAR_problem'] = pretrain['Problem Name'].apply(pretrain_problemCFAR)

pretest['CFAR_problem'] = pretest['Problem Name'].apply(problem_CFAR)

In [46]:
#caculate kcCFAR

kcCFAR = pretrain.groupby('KC(Default)')['Correct First Attempt'].agg(np.mean)

def kc_CFAR(x):
    if not pd.isnull(x):
        return kcCFAR[x]
    else:
        return kcCFAR.mean()
test['CFAR_kc'] = test['KC(Default)'].apply(kc_CFAR)

pretrain['CFAR_kc'] = pretrain['KC(Default)'].apply(kc_CFAR)

pretest['CFAR_kc'] = pretest['KC(Default)'].apply(kc_CFAR)

In [47]:
#deal with OP 
def deal_with_op(x:pd.DataFrame)->pd.DataFrame:
    newmin = []
    newnorm = []
    copy_ver = x.copy()
    result = x.copy()
    copy_ver['Opportunity(Default)'] = copy_ver['Opportunity(Default)'].astype('str')
    for i in copy_ver['Opportunity(Default)']:
        if i != 'nan':
            opsum = 0
            opmin = float('inf')
            opcal = i.split('~~')
            for j in opcal:
                if opmin > int(j):
                    opmin = int(j)
                opsum += int(j)
            opsum /= len(opcal)
            newmin.append(opmin)
            newnorm.append(opsum)
        else:
            newmin.append(0)
            newnorm.append(0)
    result['Op_min'] = newmin
    result['Op_nrom'] = newnorm
    return result

test = deal_with_op(test)

pretrain = deal_with_op(pretrain)

pretest = deal_with_op(pretest)

In [48]:
#now,deal with kc
def deal_with_kc(x:pd.DataFrame)->pd.DataFrame:
    kcnum = []
    copy_ver = x.copy()
    result = x.copy()
    copy_ver['KC(Default)'] = copy_ver['KC(Default)'].astype('str')
    for KC in copy_ver['KC(Default)']:
        if KC != 'nan':
            KCcal = KC.split('~~')
            kcnum.append(len(KCcal))
        else:
            kcnum.append(0)       
    result['KC_number'] = kcnum
    return result

test = deal_with_kc(test)

pretrain = deal_with_kc(pretrain)

pretest = deal_with_kc(pretest)

In [49]:
#delete "Problem Hierarchy", "ProblemName" and "Step Name"

def drop_lables(x):
    result = x.drop(['Problem Hierarchy','Problem Name','Step Name','Anon Student Id','Problem View',\
        'Op_min','KC(Default)','Opportunity(Default)'],axis=1)
    return result

train_x = drop_lables(pretrain)
train_y = np.array(train_x['Correct First Attempt']).astype(int)
del train_x['Correct First Attempt']

true_test_x = drop_lables(test)
del true_test_x['Correct First Attempt']

test_x = drop_lables(pretest)
test_y = np.array(test_x['Correct First Attempt']).astype(int)
del test_x['Correct First Attempt']
print(test_x)

      CFAR_student  CFAR_step  CFAR_problem   CFAR_kc    Op_nrom  KC_number
14        0.820012   0.755559      0.777679  0.686542  19.000000          3
15        0.820012   0.870237      0.885030  0.559564   0.000000          0
16        0.820012   0.818575      0.763982  0.775749  98.000000          1
17        0.820012   0.761905      0.741007  0.865864  70.000000          1
18        0.820012   0.404477      0.806701  0.460606  59.666667          3
...            ...        ...           ...       ...        ...        ...
1135      0.695489   0.567858      0.669725  0.817953   1.000000          1
1136      0.830189   0.649245      0.939189  0.867826  11.000000          1
1137      0.830189   0.000000      0.695876  0.559564   0.000000          0
1138      0.830189   0.737349      0.844037  0.817953   6.000000          1
1139      0.733333   0.707709      0.776978  0.586957  12.500000          2

[666 rows x 6 columns]


In [ ]:
#choose different method
def predict(method):
    accurate = 0
    if method == "decision_tree":
        model = tree.DecisionTreeClassifier()
        model = model.fit(train_x,train_y)
        prediction = model.predict(test_x).astype(int)
    elif method =="random_forest":
        model = RandomForestClassifier(n_estimators=200, criterion='gini', \
        max_depth=30, min_samples_split=0.01,n_jobs=4, random_state=7)
        model.fit(train_x,train_y)
        prediction = model.predict(test_x).astype(float)
    elif method == "logisticregression":
        model = linear_model.LogisticRegression()
        model.fit(train_x,train_y)
        prediction = model.predict(test_x).astype(float)
    elif method == "LGBMClassifier":
        model = lightgbm.LGBMClassifier(learning_rate=0.01,\
             n_estimators=200, min_child_weight=1, \
            subsample=1)
        model.fit(train_x,train_y)
        prediction = model.predict(test_x).astype(int)  
    elif method == "GradientBoostingClassifier":
        model = GradientBoostingClassifier()
        model.fit(train_x,train_y)
        prediction = model.predict(test_x).astype(float)
    print("the method is",method)
    print("the RMSE is ",np.sqrt(mean_squared_error(prediction,test_y)))

In [105]:
predict("decision_tree")
predict("random_forest")
predict("logisticregression")
predict("LGBMClassifier")
predict("GradientBoostingClassifier")

the method is decision_tree
the RMSE is  0.497742652128334
the method is random_forest
the RMSE is  0.39894756442314466
the method is logisticregression
the RMSE is  0.4191368221424547
the method is LGBMClassifier
the RMSE is  0.4045536598075262
the method is GradientBoostingClassifier
the RMSE is  0.4155390146215788


In [107]:
#write the data of randomforest into the file
towrite = []
base = test['Correct First Attempt'] 
model =RandomForestClassifier(n_estimators=200, criterion='gini', \
        max_depth=30, min_samples_split=0.01,n_jobs=4, random_state=7)
model.fit(train_x,train_y)
allCFA = model.predict(true_test_x).astype(float)

for i in range(len(base)):
    if base[i]!= 0 and base[i]!= 1:
        towrite.append(allCFA[i])
    else:
        towrite.append(base[i])
test_filepath = 'data/test.csv'
finalresult = testdata.copy()
finalresult['Correct First Attempt'] = towrite
finalresult.to_csv('test.csv',sep='\t')